# Training Pipeline
* ??

### Imports

In [1]:
import pandas as pd
import hopsworks
import weather_utils
from xgboost import XGBRegressor
import pickle

/Users/marcopellegrino/miniconda3/envs/myenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 1) Connect to Hopsworks Feature Store

In [2]:
# Login to Hopsworks
project = hopsworks.login()

# Retrieve Feature Store
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/178324
Connected. Call `.close()` to terminate connection gracefully.


In [3]:
# Retrieve Feature Group
weather_fg = fs.get_or_create_feature_group(
    name = 'weather_fg',
    version = 1
)

# 2) Feature View Creation and Retrieving